[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PicselliaTeam/picsellia-training-engine/blob/master/Picsellia-Yolov5-Training.ipynb)

In [ ]:
!pip install picsellia
!pip install picsellia_yolov5

In [ ]:
from picsellia.client import Client
from picsellia_yolov5.utils import to_yolo, find_matching_annotations, edit_model_yaml, generate_yaml, Opt, setup_hyp
from picsellia_yolov5.utils import send_run_to_picsellia
from picsellia_yolov5.yolov5.train import train
import argparse 
import sys
import os 
import subprocess 
import yaml 
import torch

In [ ]:
api_token = ""
project_token = ""
experiment_name = ""

In [ ]:
experiment = Client.Experiment(api_token=api_token, project_token=project_token)
exp = experiment.checkout(experiment_name, tree=True, with_file=True)
exp.dl_annotations()
exp.dl_pictures()
exp.generate_labelmap() 
exp.log('labelmap', exp.label_map, 'labelmap')

In [ ]:
YOLODIR = 'YOLO-{}'.format(experiment_name)
train_set, test_set = to_yolo(pxl_annotations_dict=exp.dict_annotations,labelmap=exp.label_map, base_imgdir=exp.png_dir, targetdir=YOLODIR, prop=0.7, copy_image=False) 

train_split = {
    'x': train_set["categories"],
    'y': train_set["train_repartition"],
    'image_list': train_set["image_list"],
}
experiment.log('train-split', train_split, 'bar', replace=True)

test_split = {
    'x': test_set["categories"],
    'y': test_set["train_repartition"],
    'image_list': test_set["image_list"],
}
experiment.log('test-split', test_split, 'bar', replace=True)

In [ ]:
generate_yaml(yamlname=experiment_name, targetdir=YOLODIR, labelmap=exp.label_map)
cfg = edit_model_yaml(label_map=exp.label_map, experiment_name=experiment_name, config_path=exp.config_dir)

In [ ]:
hyp, opt, device = setup_hyp(experiment_name, cfg, exp.checkpoint_dir, exp.get_data('parameters'), exp.label_map)

In [ ]:
train(hyp, opt, opt.device, pxl=exp)

In [ ]:
send_run_to_picsellia(exp, YOLODIR)